In [16]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [17]:
import spacy
nlp = spacy.load('en_core_web_lg')

##### move to spacy and do lematization later

In [18]:
import numpy as np
import pandas as pd

df1 = pd.read_csv('Cleaned-P&P data.csv')

blanks = []  # start with an empty list`

for i,c,d,inc,sd in df1.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks.append(i)     # add matching index numbers to the list
df1.drop(blanks, inplace=True)
df1["desc"] = df1["short_des"] + '. ' + df1["desc"]
df1.reset_index(drop=True,inplace=True)


In [19]:

df2 = pd.read_csv('Cleaned-P&P data.csv')
blanks2 = []  # start with an empty list`

for i,c,d,inc,sd in df2.itertuples():  # iterate over the DataFrame
        if c == 'O' or c=='E':         # test 'review' for whitespace
            blanks2.append(i)     # add matching index numbers to the list
df2.drop(blanks2, inplace=True)
df2.reset_index(drop=True,inplace=True)

In [20]:
dataset = pd.concat([df1, df2], ignore_index=True)

In [21]:
len(dataset)

488

In [22]:
# Cleaning the texts
import re
corpus = []
all_words = []
max_len=0
for i in range(0, 488):
    review = re.sub('[^a-zA-Z]', ' ', dataset['desc'][i])
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    
    review = nlp(review)
    review = [word.text for word in review]
    all_words.append(review) 
    if len(review) > max_len:
        max_len = len(review)
    ds = ' '.join(review)
#    for word2vec we want an array of vectors
    corpus.append(ds)
    

In [23]:
print(max_len)

114


In [24]:
all_words

[['y',
  'good',
  'afternoon',
  'this',
  'customer',
  'is',
  'being',
  'charged',
  'for',
  'delivery',
  'after',
  'purchasing',
  'the',
  'subscription',
  'please',
  'can',
  'you',
  'see',
  'if',
  'it',
  'needs',
  'activating',
  'thank',
  'you',
  'in',
  'advance',
  'rebecca',
  'connolly',
  'proactive',
  'notification',
  'team'],
 ['delivery',
  'pass',
  'needs',
  'removing',
  'on',
  'staff',
  'accounts',
  'orders',
  'supra',
  'dba',
  'req',
  'please',
  'can',
  'you',
  'remove',
  'the',
  'delivery',
  'pass',
  'be',
  'removed',
  'from',
  'the',
  'attached',
  'staff',
  'accounts',
  'thanks'],
 ['delivery',
  'pass',
  'needs',
  'removing',
  'on',
  'staff',
  'accounts',
  'orders',
  'supra',
  'dba',
  'req',
  'please',
  'can',
  'the',
  'delivery',
  'pass',
  'be',
  'removed',
  'from',
  'the',
  'attached',
  'staff',
  'accounts',
  'thanks'],
 ['customer',
  'being',
  'charged',
  'for',
  'delivery',
  'customer',
  'd',


In [25]:
all_words = [j for sub in all_words for j in sub] 

In [26]:
unique_words = set(all_words)
print(len(unique_words))
vocab_length = len(unique_words) + 10
vocab_length

752


762

In [119]:
dataset.head()

,class,desc,inc_num,short_des,Class Label
0,P,"y2684992. good afternoon,\r\n\r\nthis customer...",INC0809260,y2684992,1
1,R,delivery pass needs removing on staff accounts...,INC0808628,delivery pass needs removing on staff accounts...,0
2,R,delivery pass needs removing on staff accounts...,INC0807253,delivery pass needs removing on staff accounts...,0
3,P,customer being charged for delivery. customer:...,INC0806836,customer being charged for delivery,1
4,P,website/web shop issues. short description:tam...,INC0805994,website/web shop issues,1


In [120]:
dataset['inc_num'][49]

'INC0758972'

In [27]:
len(corpus[49])

684

In [28]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
len(embedded_sentences[49])

114

In [29]:
padded_sentences = pad_sequences(embedded_sentences, max_len, padding='post')


In [23]:
sumi=0
tot=0
for i in padded_sentences:
    for j in i:
        tot+=1
        if j == 0:
            sumi+=1
            
print(sumi, tot)

141 55632


In [162]:
# from sklearn.preprocessing import Normalizer
# transformer = Normalizer().fit(padded_sent)
# padded_sentences = transformer.transform(padded_sent)

In [30]:
len(padded_sentences[0])

114

In [173]:
model = Sequential()
model.add(Embedding(vocab_length,30, input_length=max_len))
model.add(Flatten())
# model.add(Dense(vocab_length, input_dim=30, activation='softmax'))
# model.add(Dense(vocab_length, input_dim=30, activation='softmax'))
model.add(Dense(2, activation='softmax'))

In [174]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 114, 30)           22860     
_________________________________________________________________
flatten_7 (Flatten)          (None, 3420)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 6842      
Total params: 29,702
Trainable params: 29,702
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
cldict = {'P':1 , 'R':0}
dataset['Class Label'] = dataset['class'].map(cldict)
label_series = pd.Series(dataset['Class Label'])

In [32]:
from keras.utils import to_categorical

class_label = np.array(label_series)
y = class_label
y = to_categorical(y)
y.shape

(488, 2)

In [33]:
X = padded_sentences

In [55]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [178]:
model.fit(X_train, y_train, epochs=300, verbose=0)

In [60]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

98/98 [==============================] - 0s 92us/step
Accuracy: 40.816327


In [180]:
model.predict_classes(X_train)


array([0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1,

In [1]:
model.save('P&Plarge.h5')

NameError: name 'model' is not defined

In [57]:
from keras.models import load_model
model = load_model('P&Plarge.h5')

In [13]:
#strin = 'please add the delivery subscription'
# pad = re.sub('[^a-zA-Z]', ' ', pad)
vocab_length = 762
max_len = 114
pad = strin.lower()
pad = pad.split()
pad = ' '.join(pad)
pad = nlp(pad)
pad = [word.text for word in pad]
pad = ' '.join(pad)
emb = [one_hot(pad, vocab_length)]
pady = pad_sequences(emb, max_len, padding='post')

x = model.predict_classes(pady)
print(x)
if x[0] == 1:
    print('P', model.predict(pady))
    
else: 
        print('R', model.predict(pady))
    

[1]
P [[0.2883282 0.7116718]]


In [12]:
strin = 'delivery pass needs removing on orders supra dba req please can the delivery pass be removed from the attached staff accounts thank you'

In [46]:
len(X_test)

122